In [2]:
!pip install -q sentence-transformers chromadb google-genai cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.8 MB/s eta

# Week 4 – Grant Matching App

Files expected in Colab



*   Extracted_Details.xlsx - structured grant details
*   sample_company.json - raw company descriptions







# 1.Initial setup and grant ingestion script (combine text fields + store embeddings + metadata)

In [9]:


import os, json, time
from datetime import datetime

import numpy as np
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer

import cohere


# Read Cohere key from Colab secrets or env var
try:
    from google.colab import userdata
    COHERE_API_KEY = userdata.get("COHERE_API_KEY")
except Exception:
    COHERE_API_KEY = None

co = cohere.ClientV2(api_key=COHERE_API_KEY)

LLM_MODEL = "command-r-plus-08-2024"
TODAY     = datetime.utcnow().strftime("%Y-%m-%d")

# Instruction-based embedding model
EMBED_MODEL_NAME = "hkunlp/instructor-base"
embedder = SentenceTransformer(EMBED_MODEL_NAME)

# Chroma vector DB
chroma_client = chromadb.EphemeralClient()
collection = chroma_client.get_or_create_collection(
    name="grants_collection",
    metadata={"hnsw:space": "cosine"},
)

def call_llm_json_with_cohere(prompt: str):
    """
    Call Cohere chat model and parse JSON output.
    The prompt MUST say 'return ONLY JSON'.
    """
    resp = co.chat(
        model=LLM_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    text = resp.message.content[0].text
    return json.loads(text)


grants_df = pd.read_excel("/content/Extracted_Details.xlsx")

# Text fields for semantic representation
TEXT_COLS = [
    "Funding program",
    "short Description",
    "Eligible Applicants",
    "Eligible Activities",
    "Long Description",
    "summary",
    "sector",
    "activities",
    "business_type",
]

for col in TEXT_COLS:
    if col not in grants_df.columns:
        grants_df[col] = ""
    else:
        grants_df[col] = grants_df[col].fillna("")

grants_df["grant_text"] = grants_df[TEXT_COLS].agg(" ".join, axis=1)

# Metadata
META_COLS = [
    "ID",
    "sector",
    "percentage",
    "amount",
    "country",
    "region",
    "min_year",
    "max_year",
    "min_employees",
    "max_employees",
    "activities",
    "business_type",
    "ip",
    "deadline_x",
    "deadline_y",
    "condition",
    "Funding program",
    "Grand link",
    "Region",
]
META_COLS = [c for c in META_COLS if c in grants_df.columns]

meta_df = grants_df[META_COLS].copy()

def clean_metadata_row(row: dict) -> dict:
    clean = {}
    for k, v in row.items():
        if pd.isna(v):
            continue

        if isinstance(v, (np.bool_, bool)):
            clean[k] = bool(v)
        elif isinstance(v, (np.integer, int)):
            clean[k] = int(v)
        elif isinstance(v, (np.floating, float)):
            clean[k] = float(v)
        elif isinstance(v, (pd.Timestamp, datetime)):
            clean[k] = v.isoformat()
        else:
            clean[k] = str(v)
    return clean

metadatas = [
    clean_metadata_row(row)
    for row in meta_df.to_dict(orient="records")
]

documents = grants_df["grant_text"].tolist()
ids       = grants_df["ID"].astype(str).tolist()

print("Computing INSTRUCTOR embeddings for grants--------------------------")
grant_inputs = [
    ["Represent the grant for matching with company funding needs:", text]
    for text in documents
]
grant_embs = embedder.encode(
    grant_inputs,
    batch_size=16,
    convert_to_numpy=True,
    show_progress_bar=True,
    normalize_embeddings=True,
)

collection.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas,
    embeddings=grant_embs.tolist(),
)

print(f"Grants stored in vector DB----------------------: {collection.count()}")

/tmp/ipython-input-3641391474.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  TODAY     = datetime.utcnow().strftime("%Y-%m-%d")


Computing INSTRUCTOR embeddings for grants--------------------------


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Grants stored in vector DB----------------------: 194


# 2.Load company JSON and extract structured features (Tool 1)

In [10]:


with open("/content/sample_company.json", "r", encoding="utf-8") as f:
    companies_raw = json.load(f)
company_df = pd.DataFrame(companies_raw)

company_extract_prompt = """
You are a data extraction assistant.

You will receive a JSON object called "company_detail" with keys:
- description
- industry/sector
- region
- country
- pitch_deck_extracted

From this, extract these fields and return ONLY JSON with:
{
  "sector": string or list,
  "amount_millions": number or null,
  "percentage": number or null,
  "country": string or null,
  "region": string or null,
  "years_operating": number or null,
  "employees": number or null,
  "business_type": string or null,
  "used_for": string or null,
  "priorities": array of strings
}

Guidelines:
- "amount_millions" = total funding requested expressed in millions (e.g., 0.75).
- "percentage" = percentage of project cost or eligible expenses to be covered
  (e.g., 60 for 60% coverage), if stated.
- "priorities": capture attributes like women-led, youth-led, Indigenous-led,
  rural, minority-owned, climate/sustainability focus, innovation/R&D, digital/AI/tech, etc.
- Use null or [] when something is not clearly stated.
- Do not add extra keys or any explanation text.
"""

def extract_company_features(company_row: dict) -> dict:
    payload = {"company_detail": company_row}
    prompt = company_extract_prompt + "\n\ncompany_detail:\n" + json.dumps(
        payload, ensure_ascii=False
    )

    features = call_llm_json_with_cohere(prompt)
    print("extracted company features--------------:", features)
    return features

# 3.Hybrid retrieval (Instructor + metadata filter) – Tool 2

In [11]:

def build_metadata_filter(features: dict):
    """
    Build Chroma 'where' filter using required fields:
      - country
      - region
      - years_operating  -> min_year / max_year band
      - employees        -> min_employees / max_employees band
      - amount_millions  -> amount >= requested
      - percentage       -> percentage >= requested

    Any field that is missing / None is skipped.
    If no clauses are added, return None to use semantic-only search.
    """
    clauses = []

    # country
    country = features.get("country")
    if country:
        clauses.append({"country": country})

    # region
    region = features.get("region")
    if region:
        clauses.append({"region": region})

    # years - between min_year and max_year
    years = features.get("years_operating")
    if years is not None:
        clauses.append({"min_year": {"$lte": years}})
        clauses.append({"max_year": {"$gte": years}})

    # employees - between min_employees and max_employees
    employees = features.get("employees")
    if employees is not None:
        clauses.append({"min_employees": {"$lte": employees}})
        clauses.append({"max_employees": {"$gte": employees}})

    # amount
    amount = features.get("amount_millions")
    if amount is not None:
        clauses.append({"amount": {"$gte": amount}})

    # percentage coverage
    perc = features.get("percentage")
    if perc is not None:
        clauses.append({"percentage": {"$gte": perc}})

    if not clauses:
        return None
    if len(clauses) == 1:
        return clauses[0]
    return {"$and": clauses}

def build_company_embedding_text(raw_company: dict, features: dict) -> str:
    desc  = raw_company.get("description", "")
    pitch = raw_company.get("pitch_deck_extracted", "")
    features_json = json.dumps(features, ensure_ascii=False)
    return f"{desc}\n\nPITCH:\n{pitch}\n\nSTRUCTURED_COMPANY_JSON:\n{features_json}"

def semantic_retrieve_grants(raw_company: dict, features: dict, top_k: int = 30) -> dict:
    """
    Tool 2: semantic retrieval over grants.

    Strategy:
      1) Embed combined company text + structured JSON with INSTRUCTOR.
      2) Query Chroma with FULL metadata filters (country, region, years, employees, amount, percentage).
      3) If no results:
           - Retry with COUNTRY-ONLY filter (if country is available).
      4) If still no results:
           - Fallback to PURE semantic search (no metadata filter).

    Returns the Chroma query result dict.
    """
    # 1) Build combined text for embedding
    query_text = build_company_embedding_text(raw_company, features)
    print("Query text------------------:", query_text)
    query_input = [
        "Represent the company funding needs for matching with relevant grants:",
        query_text,
    ]
    print("Query input----------------------:", query_input)
    q_emb = embedder.encode(
        [query_input],
        convert_to_numpy=True,
        normalize_embeddings=True,
    )[0].tolist()

    # 2) First try: FULL metadata filter
    where_full = build_metadata_filter(features)
    print("where_full---------------------:",where_full)
    result = collection.query(
        query_embeddings=[q_emb],
        where=where_full,
        n_results=top_k,
        include=["documents", "metadatas", "distances"],
    )

    docs = result["documents"][0]
    if len(docs) > 0:
        return result

    # 3) Second try: COUNTRY-ONLY filter (if we have a country)
    country = features.get("country")
    if country:
        print("No grants found with full filters; retrying with country-only filter.")
        where_country = {"country": country}
        result_country = collection.query(
            query_embeddings=[q_emb],
            where=where_country,
            n_results=top_k,
            include=["documents", "metadatas", "distances"],
        )
        docs_country = result_country["documents"][0]
        if len(docs_country) > 0:
            return result_country

    # 4) Final fallback: PURE semantic search (no metadata filter)
    print("No grants found with full or country-only filters; falling back to semantic-only search.---------------")
    result_semantic = collection.query(
        query_embeddings=[q_emb],
        where=None,
        n_results=top_k,
        include=["documents", "metadatas", "distances"],
    )

    return result_semantic





# 4.Cohere judge – fit score + explanation (Tool 3)

In [12]:

judge_system_prompt = f"""
You are a grant-matching analyst. Today's date is {TODAY}.

You will receive a JSON object with:
- company_detail: structured JSON describing a company.
- grant_candidates: a list of grants. Each grant has:
    - grant_id
    - grant_metadata (from Extracted_Details.xlsx)
    - grant_text (combined description text)

For EACH grant, evaluate how suitable it is for the company.

For every grant, output an object:
- "grant_id": same as input
- "fit_score": float in [0,1] (0 = not suitable, 1 = perfect fit)
- "eligibility": one of:
    "Strongly eligible", "Likely eligible",
    "Needs manual check", "Unlikely/Not eligible"
- "reasons": 1–3 sentences explaining the match/mismatch
- "risks": short note on risks (deadlines, funding mismatch, unclear eligibility, missing data, etc.)

When computing fit_score, consider:
- sector and core operations alignment
- activities / used_for vs eligible activities
- company business_type vs eligible applicants
- country and region
- priorities (e.g., women-led, Indigenous-led, rural, minority-owned, climate/sustainability, innovation)
- funding amount / percentages (amount, percentage fields)
- temporal validity (deadline_x / deadline_y vs today's date)
- missing or incomplete data (lower the score and mention uncertainty).

Return ONLY JSON (no markdown, no comments). Format:
[
  {{
    "grant_id": "...",
    "fit_score": 0.83,
    "eligibility": "Likely eligible",
    "reasons": "...",
    "risks": "..."
  }},
  ...
]
"""

def build_judge_payload(features: dict, search_result: dict, top_k: int = 10) -> dict:
    ids   = search_result["ids"][0][:top_k]
    docs  = search_result["documents"][0][:top_k]
    metas = search_result["metadatas"][0][:top_k]

    candidates = []
    for gid, text, meta in zip(ids, docs, metas):
        candidates.append({
            "grant_id": gid,
            "grant_metadata": meta,
            "grant_text": text,
        })

    return {
        "company_detail": features,
        "grant_candidates": candidates,
    }

def judge_and_rank(features: dict, search_result: dict, top_k: int = 10) -> pd.DataFrame:
    """
    Tool 3: LLM judge using Cohere.
    """
    payload = build_judge_payload(features, search_result, top_k=top_k)
    payload_str = json.dumps(payload, ensure_ascii=False)

    full_prompt = judge_system_prompt + "\n\nINPUT_JSON:\n" + payload_str

    results = call_llm_json_with_cohere(full_prompt)

    # Normalize to list
    if isinstance(results, dict):
        results_list = [results]
    else:
        results_list = results

    if not results_list:
        return pd.DataFrame(columns=["grant_id", "fit_score", "eligibility", "reasons", "risks"])

    df = pd.DataFrame(results_list)

    if "fit_score" not in df.columns:
        print("Model output did not contain 'fit_score'. Raw JSON:", results_list)
        raise ValueError("Model output missing 'fit_score'.")

    df["fit_score"] = df["fit_score"].astype(float)
    df = df.sort_values("fit_score", ascending=False).reset_index(drop=True)
    return df

# 5.Match pipeline

In [13]:

def match_company_from_raw(raw_company: dict,
                           search_k: int = 30,
                           judge_k: int = 10,
                           as_json: bool = False):
    """
    1) extract_company_features(raw_company)          [Tool 1]
    2) hybrid_retrieve_grants(raw_company, features)[Tool 2]
    3) judge_and_rank(features, search_result)        [Tool 3]

    Returns:
      - DataFrame by default
      - or list of JSON objects if as_json=True
    """
    features      = extract_company_features(raw_company)
    search_result = semantic_retrieve_grants(raw_company, features, top_k=search_k)
    ranked        = judge_and_rank(features, search_result, top_k=judge_k)
    ranked = ranked[["grant_id", "fit_score", "eligibility", "reasons", "risks"]]

    if as_json:
        return ranked.to_dict(orient="records")
    return ranked



# 6.Demo

In [15]:

sample_company_raw = {
    "description": "AgriFuture Solutions Inc. is a small-to-medium enterprise (SME) based in Ontario. We are developing AI-driven crop monitoring systems to optimize irrigation and reduce water waste. We are seeking $500,000, which is 60% of our pilot project cost, to scale field trials. We are a for-profit company operating for 2 years with 15 employees.",
    "industry/sector": "Agriculture Technology",
    "region": "Ontario",
    "country": "Canada",
    "pitch_deck_extracted": "Our technology uses proprietary drone-based thermal imaging combined with a machine learning model. We have proven a 30% reduction in water usage in our initial test plots. Funding will be used for 10 more trial deployments and to hire 2 data scientists."
  }

print("Raw company input (from sample_company.json)-----------------------------")
print(json.dumps(sample_company_raw, indent=2))

matches_df = match_company_from_raw(
    sample_company_raw,
    search_k=5,
    judge_k=2,
    as_json=False,
)

print("#######################Top Matched Grants(Final output – Dataframe)###########################")
display(matches_df)


matches_json = matches_df.to_dict(orient="records")
print("####################Top Matched Grants (Final output – Json)##########################")
print(json.dumps(matches_json, indent=2, ensure_ascii=False))


Raw company input (from sample_company.json)-----------------------------
{
  "description": "AgriFuture Solutions Inc. is a small-to-medium enterprise (SME) based in Ontario. We are developing AI-driven crop monitoring systems to optimize irrigation and reduce water waste. We are seeking $500,000, which is 60% of our pilot project cost, to scale field trials. We are a for-profit company operating for 2 years with 15 employees.",
  "industry/sector": "Agriculture Technology",
  "region": "Ontario",
  "country": "Canada",
  "pitch_deck_extracted": "Our technology uses proprietary drone-based thermal imaging combined with a machine learning model. We have proven a 30% reduction in water usage in our initial test plots. Funding will be used for 10 more trial deployments and to hire 2 data scientists."
}
extracted company features--------------: {'sector': 'Agriculture Technology', 'amount_millions': 0.5, 'percentage': 60, 'country': 'Canada', 'region': 'Ontario', 'years_operating': 2, 'em

,grant_id,fit_score,eligibility,reasons,risks
0,7,0.75,Likely eligible,The grant aligns with the company's sector and...,"The grant's deadline has passed, and the compa..."
1,99,0.40,Unlikely/Not eligible,While the grant is open to for-profit companie...,The company's business type and sector may not...


####################Top Matched Grants (Final output – Json)##########################
[
  {
    "grant_id": "7",
    "fit_score": 0.75,
    "eligibility": "Likely eligible",
    "reasons": "The grant aligns with the company's sector and region, focusing on agriculture technology and Ontario-based businesses. The company's interest in field trials and hiring aligns with the grant's emphasis on advanced automation and labor-related improvements. The grant's eligibility criteria for business type and employee count are also met by the company.",
    "risks": "The grant's deadline has passed, and the company's funding amount requested is lower than the grant's maximum, which may impact the chances of a successful application."
  },
  {
    "grant_id": "99",
    "fit_score": 0.4,
    "eligibility": "Unlikely/Not eligible",
    "reasons": "While the grant is open to for-profit companies, the company's sector and core operations do not align with the grant's focus on life sciences, health te